<font color='red'>Important NOTE: Fake Useragent Library do not work on versions superior of Python 3.6 </font>

# We will take all our client database and throught web-scrapping will find out their business activity.

Using Selenium and scrapping einforma.com

In [1]:
import pandas as pd
import numpy as np

First need to define the concrete vat numbers that need to be scrapped. Will use our merged_clean.csv file and take unique vat numbers

In [2]:
df = pd.read_csv('../data2/processed/merged_clean.csv', index_col=0)

In [3]:
df.head(2)

,id,vat,category,value,quantity
0,125,A28415495,Label Printer,239.0,1.0
1,139,B12026183,Accessories,43.8,1.0


In [ ]:
cifs = df['vat']
cifs = cifs.reset_index()

# Importing libraries needed for the webscrapping

In [ ]:
import selenium
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
#Importing the library that not works on superior version of Python 3.6
from fake_useragent import UserAgent

In [ ]:
def einforma_scrap(df):
    
    URL = 'https://www.einforma.com/servlet/app/prod/ETIQUETA_EMPRESA/nif/'
    
    #Create new columns to score info get from the scrapping
    df['Localidad'] = np.nan
    df['Actividad Informa'] = np.nan
    df['Objeto Social'] = np.nan
    df['Forma Juridica'] = np.nan
    df['CNAE'] = np.nan
    
    #Chrome User Agent
    options = Options()
    options.add_argument("window-size=800,800")

    ua = UserAgent()
    a = ua.random
    user_agent = ua.random
    print(user_agent)
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=options)
    
    total_cif = 0
    total_localidad = 0
    total_actividad = 0
    total_objeto = 0
    total_forma = 0
    bucle = 0
    no_disponible = 0
    reinicio = 0


    for i in range(len(df)):
        bucle += 1
        #print(f'bucle: {bucle}')
        driver.get(URL+ df.iloc[i]['vat'])
        time.sleep(0)
        try:
            driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/p').text == 'Servicio No Disponible'
            reinicio += 1
            
            ua = UserAgent()
            a = ua.random
            options = Options()
            options.add_argument("window-size=800,800")
            user_agent = ua.random
            print(user_agent)
            options.add_argument(f'user-agent={user_agent}')
            driver = webdriver.Chrome(options=options)
            time.sleep(0.3)
            driver.get(URL+ df.iloc[i]['vat'])
            try:
                tabla = driver.find_element_by_class_name('ficha-empresa').text
                tabla = tabla.split('\n')
                for s in range(len(tabla)):
                    tabla[s] = tabla[s].split(':')

                for f in range(len(tabla)):
                     if tabla[f][0] == 'CNAE 2009':
                        df.loc[i,'CNAE'] = (tabla[f][1])
                        total_cif +=1
                        #print(f'total cif = {total_cif}')

                for f in range(len(tabla)):
                     if tabla[f][0] == 'Localidad':
                        df.loc[i,'Localidad'] = (tabla[f][1])
                        total_localidad +=1
                        #print(f'total loc = {total_localidad}')

                for f in range(len(tabla)):
                     if tabla[f][0] == 'Actividad Informa':
                        df.loc[i,'Actividad Informa'] = (tabla[f][1])
                        total_actividad +=1
                        #print(f'total act = {total_actividad}')

                for f in range(len(tabla)):
                     if tabla[f][0] == 'Objeto Social':
                        df.loc[i,'Objeto Social'] = (tabla[f][1])
                        total_objeto +=1
                        #print(f'total obj = {total_objeto}')                   

                for f in range(len(tabla)):
                     if tabla[f][0] == 'Forma Jurídica':
                        df.loc[i,'Forma Juridica'] = (tabla[f][1])
                        total_forma +=1
                        #print(f'total form = {total_forma}')  

            except:
                no_disponible += 1
                #print(f'Cuenta no encontrado: {no_disponible}')
                pass

        except:

            try:
                tabla = driver.find_element_by_class_name('ficha-empresa').text
                tabla = tabla.split('\n')
                for s in range(len(tabla)):
                    tabla[s] = tabla[s].split(':')

                for f in range(len(tabla)):
                     if tabla[f][0] == 'CNAE 2009':
                        df.loc[i,'CNAE'] = (tabla[f][1])
                        total_cif +=1
                        #print(f'total cif = {total_cif}')

                for f in range(len(tabla)):
                     if tabla[f][0] == 'Localidad':
                        df.loc[i,'Localidad'] = (tabla[f][1])
                        total_localidad +=1
                        #print(f'total loc = {total_localidad}')

                for f in range(len(tabla)):
                     if tabla[f][0] == 'Actividad Informa':
                        df.loc[i,'Actividad Informa'] = (tabla[f][1])
                        total_actividad +=1
                        #print(f'total act = {total_actividad}')

                for f in range(len(tabla)):
                     if tabla[f][0] == 'Objeto Social':
                        df.loc[i,'Objeto Social'] = (tabla[f][1])
                        total_objeto +=1
                        #print(f'total obj = {total_objeto}')                   

                for f in range(len(tabla)):
                     if tabla[f][0] == 'Forma Jurídica':
                        df.loc[i,'Forma Juridica'] = (tabla[f][1])
                        total_forma +=1
                        #print(f'total form = {total_forma}')  

            except:
                no_disponible += 1
                #print(f'Cuenta no encontrado: {no_disponible}')
                pass
    
    

In [ ]:
einforma_scrap(cifs)

In [ ]:
cifs.to_csv('../data2/processed/final_cifs_enrich.csv')